In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as op
from scipy.optimize import curve_fit
import math
import torch

In [2]:
def hardsigmoid(x, a, b, c, d):
    linear_segment = a + (b - a) / (d - c) * (x - c)
    return np.where(x < c, a, np.where(x <= d, linear_segment, b))


In [3]:
data = torch.load('./data/hard_sigmoid_vout.data').numpy()
data.shape

(401, 9999)

In [5]:
x = np.linspace(-2, 2, 401)
result = []

for k in range(data.shape[1]):
    y = data[:, k]
    
    popt, _ = curve_fit(hardsigmoid, x, y, maxfev=1000000, p0=[0., 1., 0., 1.], bounds=([-np.inf, -np.inf, -np.inf, -np.inf], [np.inf, np.inf, np.inf, np.inf]))
    eta_1, eta_2, eta_3, eta_4 = popt
    result.append([eta_1, eta_2, eta_3, eta_4])

    plt.figure()
    plt.plot(x, y, color='green')
    plt.plot(x, hardsigmoid(x, *popt), color='red') 
    plt.xlim(-2, 2)
    plt.savefig(f'./eta_figures/{k}.pdf', format='pdf', bbox_inches='tight')
    plt.close()

In [5]:
a = torch.tensor(result)
a.shape

torch.Size([9999, 4])

In [6]:
torch.save(a, './data/hard_sigmoid_param.data')